In [31]:
# ÉTAPE 1 — Conversion TSV -> CSV (par chunks, mémoire safe)

import os
import pandas as pd

# === À ADAPTER ===
DATA_DIR   = "data/imdb"    # <-- mets ton chemin
OUTPUT_DIR = os.path.join(DATA_DIR, "csv_out")
CHUNK_SIZE = 500_000                   # nb de lignes par chunk (ajuste si besoin)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Fichiers attendus (convertira seulement ceux trouvés)
FILES = [
    "title.basics.tsv",
    "title.akas.tsv",
    "title.crew.tsv",
    "title.episode.tsv",
    "title.principals.tsv",
    "name.basics.tsv",
    "title.ratings.tsv"
]

NA_TOKEN = r"\N"

def tsv_to_csv(tsv_path: str, csv_path: str, chunk_size: int = CHUNK_SIZE):
    print(f"\n-> Conversion: {os.path.basename(tsv_path)}")
    first = True
    # lecture par chunks pour limiter la RAM
    for i, chunk in enumerate(pd.read_csv(
        tsv_path,
        sep="\t",
        na_values=[NA_TOKEN, ""],
        keep_default_na=False,
        low_memory=False,
        chunksize=chunk_size,
        encoding="utf-8",
        on_bad_lines="skip"
    )):
        mode = "w" if first else "a"
        header = first
        chunk.to_csv(csv_path, index=False, mode=mode, header=header, encoding="utf-8")
        first = False
        print(f"  - chunk {i+1} écrit")

    print(f"OK -> {csv_path}")

for fname in FILES:
    in_path = os.path.join(DATA_DIR, fname)
    if not os.path.exists(in_path):
        print(f"(!) Introuvable, je passe: {fname}")
        continue
    out_path = os.path.join(OUTPUT_DIR, fname.replace(".tsv", ".csv"))
    tsv_to_csv(in_path, out_path)



-> Conversion: title.basics.tsv
  - chunk 1 écrit
  - chunk 2 écrit
  - chunk 3 écrit
  - chunk 4 écrit
  - chunk 5 écrit
  - chunk 6 écrit
  - chunk 7 écrit
  - chunk 8 écrit
  - chunk 9 écrit
  - chunk 10 écrit
  - chunk 11 écrit
  - chunk 12 écrit
  - chunk 13 écrit
  - chunk 14 écrit
  - chunk 15 écrit
  - chunk 16 écrit
  - chunk 17 écrit
  - chunk 18 écrit
  - chunk 19 écrit
  - chunk 20 écrit
  - chunk 21 écrit
  - chunk 22 écrit
  - chunk 23 écrit
  - chunk 24 écrit
OK -> data/imdb\csv_out\title.basics.csv

-> Conversion: title.akas.tsv
  - chunk 1 écrit
  - chunk 2 écrit
  - chunk 3 écrit
  - chunk 4 écrit
  - chunk 5 écrit
  - chunk 6 écrit
  - chunk 7 écrit
  - chunk 8 écrit
  - chunk 9 écrit
  - chunk 10 écrit
  - chunk 11 écrit
  - chunk 12 écrit
  - chunk 13 écrit
  - chunk 14 écrit
  - chunk 15 écrit
  - chunk 16 écrit
  - chunk 17 écrit
  - chunk 18 écrit
  - chunk 19 écrit
  - chunk 20 écrit
  - chunk 21 écrit
  - chunk 22 écrit
  - chunk 23 écrit
  - chunk 24 écrit
 

In [33]:
# ÉTAPE 2 — Aperçus rapides des CSV (échantillon léger)

import os
import pandas as pd

# === À ADAPTER ===
DATA_DIR   = "data/imdb/csv_out"
SAMPLE_N   = 200_000                         # lignes lues par fichier pour l'aperçu

pd.options.display.max_columns = 120
pd.options.display.width = 140

FILES = {
    "title.basics.csv":     {"extra_counts": {"titleType": 15, "isAdult": 5}},
    "title.akas.csv":       {"extra_counts": {"region": 15, "language": 15}},
    "name.basics.csv":      {"extra_counts": {"primaryProfession": 15}},
    "title.principals.csv": {"extra_counts": {"category": 15}},
    "title.crew.csv":       {"extra_counts": {}},
    "title.episode.csv":    {"extra_counts": {}},
    "title.ratings.csv":    {"extra_counts": {}},     # nouveau fichier
}

def quick_preview_csv(path: str, sample_n: int = SAMPLE_N, extra_counts: dict | None = None):
    name = os.path.basename(path)
    if not os.path.exists(path):
        print(f"(!) Introuvable, je passe : {name}")
        return None

    print(f"\n=== {name} ===")
    print(f"Taille fichier: {os.path.getsize(path)/1e6:.1f} Mo")

    # lecture d'un échantillon (rapide)
    df = pd.read_csv(path, nrows=sample_n, low_memory=False)
    print(f"Échantillon lu: {len(df):,} lignes  |  Colonnes: {len(df.columns)}")

    # aperçu head
    print("\nHead:")
    display(df.head(10))

    # % de valeurs manquantes (sur l'échantillon)
    na_pct = (df.isna().mean() * 100).sort_values(ascending=False)
    print("\nTaux de valeurs manquantes (échantillon) — top 20 :")
    display(na_pct.head(20).to_frame("na_%").round(2))

    # stats numériques simples
    num_cols = df.select_dtypes(include=["number"]).columns
    if len(num_cols):
        print("\nRésumé numérique (échantillon):")
        display(df[num_cols].describe().T[["count", "mean", "std", "min", "50%", "max"]])

    # value_counts utiles
    extra_counts = extra_counts or {}
    for col, topk in extra_counts.items():
        if col in df.columns:
            print(f"\nValue counts — {col} (top {topk}) :")
            display(df[col].value_counts(dropna=False).head(topk))

    return df


# ----- Lancer les aperçus -----
samples = {}
for fname, opts in FILES.items():
    path = os.path.join(DATA_DIR, fname)
    samples[fname] = quick_preview_csv(path, SAMPLE_N, opts.get("extra_counts", {}))


# ----- Petit bonus utile si ratings dispo : top des notes (échantillon) -----
if samples.get("title.ratings.csv") is not None and samples.get("title.basics.csv") is not None:
    rat = samples["title.ratings.csv"][["tconst", "averageRating", "numVotes"]].dropna()
    bas = samples["title.basics.csv"][["tconst", "primaryTitle", "startYear", "titleType"]]
    tmp = (rat.merge(bas, on="tconst", how="left")
              .query("numVotes >= 1000")  # seuil simple
              .sort_values(["averageRating","numVotes"], ascending=[False, False])
              .head(20))
    print("\nTop titres (échantillon) — note moyenne (>=1000 votes) :")
    display(tmp[["primaryTitle", "startYear", "titleType", "averageRating", "numVotes"]])



=== title.basics.csv ===
Taille fichier: 1031.0 Mo
Échantillon lu: 200,000 lignes  |  Colonnes: 9

Head:


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894.0,NaN,1.0,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892.0,NaN,5.0,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892.0,NaN,5.0,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892.0,NaN,12.0,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893.0,NaN,1.0,Short
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894.0,NaN,1.0,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894.0,NaN,1.0,"Short,Sport"
7,tt0000008,short,Edison Kinetoscopic Record of a Sneeze,Edison Kinetoscopic Record of a Sneeze,0,1894.0,NaN,1.0,"Documentary,Short"
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894.0,NaN,45.0,Romance
9,tt0000010,short,Leaving the Factory,La sortie de l'usine Lumière à Lyon,0,1895.0,NaN,1.0,"Documentary,Short"



Taux de valeurs manquantes (échantillon) — top 20 :


,na_%
endYear,95.64
runtimeMinutes,19.52
genres,8.60
startYear,0.12
tconst,0.00
isAdult,0.00
originalTitle,0.00
primaryTitle,0.00
titleType,0.00



Résumé numérique (échantillon):


...

,na_%
tconst,0.0
averageRating,0.0
numVotes,0.0



Résumé numérique (échantillon):


,count,mean,std,min,50%,max
averageRating,200000.0,6.204634,1.219757,1.0,6.3,10.0
numVotes,200000.0,2531.920060,32132.421957,5.0,65.0,3082234.0



Top titres (échantillon) — note moyenne (>=1000 votes) :


,primaryTitle,startYear,titleType,averageRating,numVotes
127283,The Legend of Zelda: Ocarina of Time,1998.0,videoGame,9.6,11431
125077,Metal Gear Solid,1998.0,videoGame,9.5,13774
139578,Final Fantasy VII,1997.0,videoGame,9.5,11818
151243,NaN,NaN,NaN,9.5,1649
127999,Band of Brothers,2001.0,tvMiniSeries,9.4,569210
176668,NaN,NaN,NaN,9.4,8615
126377,Fallout 2: A Post-Nuclear Role-Playing Game,1998.0,videoGame,9.4,4518
157042,NaN,NaN,NaN,9.4,4439
140774,NaN,NaN,NaN,9.4,3500
147635,NaN,NaN,NaN,9.4,2440


In [37]:
# ÉTAPE 3 — Stats globales sur tous les fichiers CSV
import pandas as pd
import os

DATA_DIR = "data/imdb/csv_out"  # dossier CSV

files = [
    "title.basics.csv",
    "title.akas.csv",
    "name.basics.csv",
    "title.principals.csv",
    "title.crew.csv",
    "title.episode.csv",
    "title.ratings.csv",
]

summary_stats = []

def get_file_stats(fname):
    path = os.path.join(DATA_DIR, fname)
    if not os.path.exists(path):
        return None
    # Compte des lignes rapides
    total_rows = sum(1 for _ in open(path, "r", encoding="utf-8")) - 1
    # Lecture d'un mini échantillon juste pour colonnes
    cols = pd.read_csv(path, nrows=5).columns.tolist()
    return {"fichier": fname, "lignes": total_rows, "colonnes": len(cols), "noms_colonnes": cols}

for f in files:
    stats = get_file_stats(f)
    if stats:
        summary_stats.append(stats)

df_summary = pd.DataFrame(summary_stats)
print("\n=== Récap taille fichiers ===")
display(df_summary)

# --- Quelques agrégations globales ---
# Top genres (title.basics)
tb_path = os.path.join(DATA_DIR, "title.basics.csv")
tb = pd.read_csv(tb_path, usecols=["genres"])
genres_counts = tb["genres"].dropna().str.split(",").explode().value_counts().head(20)
print("\nTop genres (global) :")
display(genres_counts)

# Top régions (title.akas)
ta_path = os.path.join(DATA_DIR, "title.akas.csv")
ta = pd.read_csv(ta_path, usecols=["region"])
region_counts = ta["region"].value_counts().head(20)
print("\nTop régions (global) :")
display(region_counts)

# Top professions (name.basics)
nb_path = os.path.join(DATA_DIR, "name.basics.csv")
nb = pd.read_csv(nb_path, usecols=["primaryProfession"])
prof_counts = nb["primaryProfession"].dropna().str.split(",").explode().value_counts().head(20)
print("\nTop professions (global) :")
display(prof_counts)

# Top catégories (title.principals)
tp_path = os.path.join(DATA_DIR, "title.principals.csv")
tp = pd.read_csv(tp_path, usecols=["category"])
cat_counts = tp["category"].value_counts().head(20)
print("\nTop catégories (global) :")
display(cat_counts)

# Meilleurs films/series par note moyenne (>= 50 000 votes)
tr_path = os.path.join(DATA_DIR, "title.ratings.csv")
tr = pd.read_csv(tr_path)
tb_small = pd.read_csv(tb_path, usecols=["tconst", "primaryTitle", "startYear", "titleType"])
ratings_merge = tr.merge(tb_small, on="tconst", how="left")
top_rated = (
    ratings_merge.query("numVotes >= 50000")
    .sort_values(["averageRating", "numVotes"], ascending=[False, False])
    .head(20)
)
print("\nTop titres (>= 50k votes) :")
display(top_rated)



=== Récap taille fichiers ===


,fichier,lignes,colonnes,noms_colonnes
0,title.basics.csv,11836175,9,"[tconst, titleType, primaryTitle, originalTitle, isAdult, startYear, endYear, runtimeMinutes, genres]"
1,title.akas.csv,52816959,8,"[titleId, ordering, title, region, language, types, attributes, isOriginalTitle]"
2,name.basics.csv,14628926,6,"[nconst, primaryName, birthYear, deathYear, primaryProfession, knownForTitles]"
3,title.principals.csv,94059043,6,"[tconst, ordering, nconst, category, job, characters]"
4,title.crew.csv,11834005,3,"[tconst, directors, writers]"
5,title.episode.csv,9111019,4,"[tconst, parentTconst, seasonNumber, episodeNumber]"
6,title.ratings.csv,1601448,3,"[tconst, averageRating, numVotes]"



Top genres (global) :


genres
Drama          3335718
Comedy         2298638
Talk-Show      1475278
Short          1259277
Documentary    1130418
News           1117888
Romance        1099258
Family          865014
Reality-TV      662679
Animation       591599
Crime           504176
Action          493995
Adventure       453584
Game-Show       447056
Music           439355
Adult           380570
Sport           300258
Fantasy         258734
Mystery         247536
Horror          236903
Name: count, dtype: int64


Top régions (global) :


...

primaryProfession
actor                   3368319
actress                 2036720
miscellaneous           1512943
producer                1294218
writer                   968765
camera_department        860445
director                 792406
art_department           515523
cinematographer          423695
sound_department         420496
editor                   390011
composer                 368139
music_department         304587
assistant_director       281364
visual_effects           265137
make_up_department       244683
animation_department     237870
production_manager       225897
archive_footage          217699
editorial_department     205278
Name: count, dtype: int64


Top catégories (global) :


category
actor                  22303385
actress                16848366
self                   13879625
writer                 11266540
director                8088856
producer                7061085
editor                  4966314
cinematographer         3782838
composer                3044674
production_designer     1118950
casting_director        1096504
archive_footage          591117
archive_sound             10789
Name: count, dtype: int64


Top titres (>= 50k votes) :


,tconst,averageRating,numVotes,titleType,primaryTitle,startYear
987038,tt2301451,10.0,256946,tvEpisode,Ozymandias,2013.0
1293980,tt4283088,9.9,242555,tvEpisode,Battle of the Bastards,2016.0
1293982,tt4283094,9.9,174426,tvEpisode,The Winds of Winter,2016.0
987040,tt2301455,9.9,163723,tvEpisode,Felina,2013.0
955247,tt2178784,9.9,133055,tvEpisode,The Rains of Castamere,2013.0
830014,tt1683088,9.9,89006,tvEpisode,Face Off,2011.0
1214399,tt34952922,9.9,78427,tvEpisode,I Thought You'd Never Shut Up,2025.0
604227,tt12187040,9.9,67358,tvEpisode,Plan and Execution,2022.0
1600954,tt9906260,9.8,150120,tvEpisode,Hero,2019.0
1267243,tt3866850,9.8,116946,tvEpisode,Hardhome,2015.0
